# Graduate 8650 Data Structures class Project 

The first three cells are for initialization. You may simply run the entire notebook and see the results. You are also encouraged to play around with e.g. the graph sizes to see different output times.

In [33]:
import csv
import timeit

In [34]:
class PriorityQueue():
    '''
    The arguments passed to a PriorityQueue must consist of
    objects than can be compared using <.
    Use a tuple (priority, item) if necessary.
    '''

    def __init__(self):
        self._array = []

    def push(self, obj):
        # append at end and bubble up
        self._array.append(obj)
        n = len(self._array)
        self._bubble_up(n-1)
        
    def pop(self):
        n = len(self._array)
        if n==0:
            return None
        if n==1:
            return self._array.pop()
        
        # replace with last item and sift down:
        obj = self._array[0]
        self._array[0] = self._array.pop()
        self._sift_down(0)
        return obj
    
    def _parent(self, n):
        return (n-1)//2

    def _left_child(self, n):
        return 2*n + 1

    def _right_child(self, n):
        return 2*n + 2

    def _bubble_up(self, index):
        while index>0:
            cur_item = self._array[index]
            parent_idx = self._parent(index)
            parent_item = self._array[parent_idx]
            
            if cur_item < parent_item:
                # swap with parent
                self._array[parent_idx] = cur_item
                self._array[index] = parent_item
                index = parent_idx
            else:
                break
    
    def _sift_down(self,index):
        n = len(self._array)
        
        while index<n:           
            cur_item = self._array[index]
            lc = self._left_child(index)
            if n <= lc:
                break

            # first set small child to left child:
            small_child_item = self._array[lc]
            small_child_idx = lc
            
            # right exists and is smaller?
            rc = self._right_child(index)
            if rc < n:
                r_item = self._array[rc]
                if r_item < small_child_item:
                    # right child is smaller than left child:
                    small_child_item = r_item
                    small_child_idx = rc
            
            # done: we are smaller than both children:
            if cur_item <= small_child_item:
                break
            
            # swap with smallest child:
            self._array[index] = small_child_item
            self._array[small_child_idx] = cur_item
            
            # continue with smallest child:
            index = small_child_idx
        
    def size(self):
        return len(self._array)
    
    def is_empty(self):
        return len(self._array) == 0
    
    def heapify(self, items):
        """ Take an array of unsorted items and replace the contents
        of this priority queue by them. """
        self._array = items
        i = len(self._array)//2
        while i != -1:
            self._sift_down(i)
            i-=1

In [35]:
def MSTKruskal(E):
    
    blue = []                            #initialize
    V = set()
    for e in E:
        V.add(e[1])
        V.add(e[2])
    forest = Partition(V)


    P = PriorityQueue()                      #place the edges in a priority queue to be able to find the edge
    P.heapify(E)                             #with smallest edge weight and to not query edges which are not 
                                             #necessary (i.e. sorting first may sort edges we will never use)
    n = len(V)
    while len(blue) != n-1 and not len(P._array) == 0:  #do until blue tree has n-1 edges or until there are no 
        l = P.pop()                                     #edges left to consider
        u = forest.find(l[1])                          
        v = forest.find(l[2])                       #for each (u,v) = e in E, in order by edge weight from
        if u != v:                                  #low to high, check if u,v are  both in the  same partition;
            forest.link(u,v)                        #if not, put them in the same partition and add (u,v)
            blue.append(l)                          #to blue. If they are, skip this edge and go to the edge with  
                                                    #the next smallest edge weight by popping from P.
    return(blue)

The first portion of the jupyter notebook is for the updated MST algorithm with the heuristics added.

In [36]:
#This is a graph extracted from a file from the Stanford Network Analysis Project, with the title of this data
#set as Bitcoin OTC trust weighted signed network. This is cited in the write up of this project.

with open('soc-sign-bitcoinotc.csv', 'r') as f:
    reader = csv.reader(f)
    l = []
    for row in reader:
        l.append((int(row[0]),row[1],row[2]))

print("Number of edges in entire graph is",len(l))
print("The first five edges in the graph are",l[:5])

#l = l[:7000]               #This line may be modified to change the number of edges in the graph to e.g. 7000
#print("The number of edges in the graph for the demonstration is",len(l)) 

#The above two lines are for encouragement to try different graph sizes. Trying the
#algorithms on e.g. 7000 edges shows the difference in compute time instead of just throwing an error
#for the naive implementation

Number of edges in entire graph is 35592
The first five edges in the graph are [(4, '2', '6'), (2, '5', '6'), (1, '15', '1'), (7, '3', '4'), (8, '16', '13')]


The following is the Partition class where the Partition class is implemented with path compression and the rank heuristic.

In [37]:
#Updated Partition class with heuristics
class Partition():
    
    def __init__(self,V):                                  #the data structure is a map, and for each node v,
        self._position = {}                                #the value of v is v's parent node and v's rank,
        for v in V:                                        #where v's parent node is initialized to v.
            self._position[v] = [v,0]  
        
    def find(self,v):                           #find finds and returns the representative of the part which
        if self._position[v][0] != v:           #contains v by following the parent node path of v to the 
            self._position[v][0] = self.find(self._position[v][0])  #representative, updating the parent 
        return(self._position[v][0])                            #of each node seen to the representative element. 
    
    def link(self,u,v):                                    #Note that link's inputs are representative elements
        if self._position[u][1] > self._position[v][1]:    #and not arbitrary elements. link takes two
            u,v = v,u                                      #representatives u,v and if rank(u) = rank(v),
        if self._position[u][1] == self._position[v][1]:   #then increase rank(v) by one and set u's parent node 
             self._position[v][1]+=1                       #v. If wlog rank(u) < rank(v), set u's parent node to  
        self._position[u][0] = v                           #v.

In [38]:

start = timeit.default_timer()
t = MSTKruskal(l);        #Checking time for Kruskal with updated Partition class
print(len(t))
print("Time for implementation with path compression and union by rank is",timeit.default_timer() - start)

5877
Time for implementation with path compression and union by rank is 0.5621557279955596


In [39]:
#A smaller example so that you may see an example MST yourself
E = [(8, 'a', 'b'), (5, 'c', 'a'), (16, 'f', 'c'), (26, 'g', 'f'), (4, 'e', 'g'), (18, 'b', 'e'), (10, 'c', 'b'), (2, 'd', 'b'), (3, 'c', 'd'), (12, 'd', 'e'), (30, 'f', 'd'), (14, 'g', 'd')]
print(MSTKruskal(E))

[(2, 'd', 'b'), (3, 'c', 'd'), (4, 'e', 'g'), (5, 'c', 'a'), (12, 'd', 'e'), (16, 'f', 'c')]


Now we use the naive partition class. Note the file must be either read in again or alternatively saved in two distinct lists at the start, as an execution of the MST algorithm removes the elements from the original list as it executes. We choose to read in the file again.

In [40]:
with open('soc-sign-bitcoinotc.csv', 'r') as f:
    reader = csv.reader(f)
    l = []
    for row in reader:
        l.append((int(row[0]),row[1],row[2]))

print("Number of edges in entire graph is",len(l))
print("The first five edges in the graph are",l[:5])

#l = l[:7000]               #This line may be modified to change the number of edges in the graph to e.g. 7000
#print("The number of edges in the graph for the demonstration is",len(l)) 

#The above two lines are for encouragement to try different graph sizes. Trying the
#algorithms on e.g. 7000 edges shows the difference in compute time instead of just throwing an error
#for the naive implementation

Number of edges in entire graph is 35592
The first five edges in the graph are [(4, '2', '6'), (2, '5', '6'), (1, '15', '1'), (7, '3', '4'), (8, '16', '13')]


The following is the Naive Partition class where the Partition class is implemented without path compression and the rank heuristic.

In [41]:
#Naive version of the partition class. Does not implement the rank heuristic or path compression

class Partition():              
                                
    def __init__(self,V):
        self._position = {}                   #data structure is still a map, but key value is only parent node
        for v in V:                           #instead of parent, rank
            self._position[v] = v
        
    def find(self,v):
        if self._position[v] == v:                #this function does not update the parent nodes along the parent
            return(v)                             #node path to be the representative element, it only follows  
        else:                                     #the parent node path and returns the representative of the  
            return(self.find(self._position[v]))  #part which contatins v.
    
    def link(self,u,v):                        #just changes representative u's parent node from u 
        self._position[u] = v                  #to the representative v

In [42]:
start = timeit.default_timer()
t = MSTKruskal(l);                              #Checking time for Kruskal with naive Parition class
print(len(t))
print("Time for mst with naive implementation is",timeit.default_timer() - start)

RecursionError: maximum recursion depth exceeded in comparison

In [43]:
#Another small example to show the MST
E = [(8, 'a', 'b'), (5, 'c', 'a'), (16, 'f', 'c'), (26, 'g', 'f'), (4, 'e', 'g'), (18, 'b', 'e'), (10, 'c', 'b'), (2, 'd', 'b'), (3, 'c', 'd'), (12, 'd', 'e'), (30, 'f', 'd'), (14, 'g', 'd')]
print(MSTKruskal(E))

[(2, 'd', 'b'), (3, 'c', 'd'), (4, 'e', 'g'), (5, 'c', 'a'), (12, 'd', 'e'), (16, 'f', 'c')]
